In [1]:
import pandas as pd
from line_profiler import LineProfiler
import numpy as np
from numba import jit

1. В файлах recipes_short.csv и reviews_short.csv находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде pd.DataFrame с названиями recipes и reviews. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.


In [2]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv')
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')

2. Реализуйте несколько вариантов функции подсчета среднего значения столбца rating из таблицы reviews для отзывов, оставленных в 2010 году.

-   С использованием метода DataFrame.iterrows и без использования срезов таблицы
-   С использованием метода DataFrame.iterrows и с использованием срезов таблицы
-   С использованием метода DataFrame.mean

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.


In [19]:
def method1():
    data = reviews[((reviews['date'] >= "2010-01-01") &
                     (reviews['date'] < "2011-01-01"))]
    s = 0
    a = 0
    for _, row in data.iterrows():
        s += row['rating']
        a += 1
    return s/a


method1()

4.4544402182900615

In [17]:
def method2():
    total = 0
    amount = 0
    for _, row in reviews.iterrows():
        if row['date'] >= "2010-01-01" and row['date'] < "2011-01-01":
            total += row['rating']
            amount += 1
    return total/amount

method2()

4.4544402182900615

In [18]:
def method3():
    data = reviews[((reviews['date'] >= "2010-01-01") &
                     (reviews['date'] < "2011-01-01"))]
    return data['rating'].mean()


method3()

4.4544402182900615

3. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер line_profiler. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.
   (\*). Сможете ли вы ускорить работу функции 2В?


In [12]:
lp = LineProfiler()
lp(method1)()
lp(method2)()
lp(method3)()
lp.print_stats()


Timer unit: 1e-07 s

Total time: 1.81433 s

Could not find file C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_5304\1541004323.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2                                           
     3         1      62400.0  62400.0      0.3  
     4         1      55411.0  55411.0      0.3  
     5         1          4.0      4.0      0.0  
     6         1          2.0      2.0      0.0  
     7     12094   15696059.0   1297.8     86.5  
     8     12094    2276528.0    188.2     12.5  
     9     12094      52926.0      4.4      0.3  
    10         1          7.0      7.0      0.0  

Total time: 0.0135256 s

Could not find file C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_5304\2857053116.py
Are you sure you are running this program from the

Самый медленный способ - тот который проверяет уловие каждый раз, быстрее, тот который единожды проверив условие считает нужное значение, т.к. фильтрация большого объема данных быстрее делается не по отдельности, а сразу, ну и самый быстрый метод - через встроенную функцию, т.к. библиотека по максимуму использует вычислительные способности компьютера не делая лишних действий

---
4. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.


In [21]:
def count_words(word):
    data = reviews['review']
    i = 0
    for review in data.fillna(''):
        for word_r in review:
            if word_r == word:
                i += 1
                break
    return i


lp = LineProfiler()
lp(count_words)('I')
lp.print_stats()


Timer unit: 1e-07 s

Total time: 6.06589 s

Could not find file C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_5304\3917346224.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1        449.0    449.0      0.0  
     3         1     192317.0 192317.0      0.3  
     4         1          6.0      6.0      0.0  
     5    126696    1109036.0      8.8      1.8  
     6   7063215   28376903.0      4.0     46.8  
     7   6957574   29887717.0      4.3     49.3  
     8    105641     577949.0      5.5      1.0  
     9    105641     514469.0      4.9      0.8  
    10         1          8.0      8.0      0.0  



Не оптимизировано прохождение по списку, из за двух вложенных `for` проходищих по огромному списку мы получаем очень медленную функцию

In [3]:
def count_words_opt(word):
    task = reviews
    return task['review'].str.contains('I', na=False).sum()

In [8]:
lp = LineProfiler()
lp(count_words_opt)('I')

lp.print_stats()

Timer unit: 1e-07 s

Total time: 0.171287 s

Could not find file C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_7592\1954451170.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2         1          7.0      7.0      0.0  
     3         1    1712866.0 1712866.0    100.0  



---
5. Напишите несколько версий функции MAPE (см. MAPE) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва.

1)  Без использования массивов numpy и numba
2)  Без использования массивов numpy, но с использованием numba
3)  С использованием массивов numpy, но без использования numba
4)  C использованием массивов numpy и numba
    Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [12]:
def mape(ratings):
    m = sum(ratings)/len(ratings)
    s = [abs(rat - m) for rat in ratings]
    return 100*sum(s)/len(s)

%timeit mape(reviews['rating'])

48.8 ms ± 3.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
mape_with_numba = jit(mape)

%timeit mape_with_numba(reviews['rating'])


C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_12500\3408752179.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  mape_with_numba = jit(mape)
C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_12500\1283778772.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "mape" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_12500\1283778772.py (1)

File "C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_12500\1283778772.py", line 1:
def mape(ratings):
^

  def mape(ratings):
c:\Programs\Python\Python311\Lib\site-packages\numba\core\object_mode_pas

108 ms ± 30.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
def mape_np(ratings):
    ratings = np.array(ratings)
    return 100*abs(ratings - ratings.mean()).mean()


%timeit mape_np(reviews['rating'])


830 µs ± 135 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [16]:
mape_np_with_numba = jit(mape_np)

%timeit mape4(reviews['rating'])


C:\Users\DaSh-More\AppData\Local\Temp\ipykernel_12500\680963382.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  mape_np_with_numba = jit(mape_np)


985 µs ± 167 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
